

what is also practical if you want to have a second look at the tweets as they are saved in twitter: advanced search tool of twitter: https://twitter.com/search-advanced?lang=en


# Import modules

In [2]:
# Base and Cleaning 
import pandas as pd
import numpy as np
from datetime import datetime
import sys

#Natural Language Processing (NLP)
import spacy     # check out 
from spacy.tokenizer import Tokenizer
import gensim
from gensim.corpora import Dictionary # check out 
from gensim import models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS # what is that? 
stopwords = set(STOPWORDS)

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls


# settings for easier coding and inspection of data  
%matplotlib notebook
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-ffbd8e905850>:31: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [3]:
import warnings
warnings.simplefilter('ignore')#"error", "ignore", "always", "default", "module"

# Import data 

In [5]:
parser = lambda x: datetime.strptime(x,'%Y-%m-%d %X') 
df_import = pd.read_csv("full_tweets_without_rt.xlsx",
                             index_col=0,parse_dates =['created_at'], date_parser=parser)


In [6]:
df_import

,agency,created_at,text,date
1340367556947943425,yahoonews,2020-12-19 18:45:03,"A $20,000 fee to change a wedding date? New bill aims to protect people from 'bad actor catering halls' https://t.co/57rja8kptk https://t.co/Yy6u10D1xf",2020-12-19 00:00:00
1340329823282016256,yahoonews,2020-12-19 16:15:06,U.S. intelligence agencies are increasingly focused on domestic extremists. Their latest target: Satanists https://t.co/pBUJGL0ZxQ https://t.co/Up0XpgKt9z,2020-12-19 00:00:00
1340307168923414531,yahoonews,2020-12-19 14:45:05,Rebuilding trust in the Justice Department starts — but doesn't end — with Biden's choice of attorney general https://t.co/JriZ0YmQKl https://t.co/mvCEfyziJV,2020-12-19 00:00:00
1340141062724194306,yahoonews,2020-12-19 03:45:02,Biden says Lindsey Graham is a 'personal disappointment' for not recognizing him as president-elect https://t.co/HONKs8lHru https://t.co/5XDdxh3L5b,2020-12-19 00:00:00
1340065561523458048,yahoonews,2020-12-18 22:45:01,75 people and counting test positive for COVID after North Carolina church’s Christmas musical https://t.co/Xk7u4lMnke https://t.co/PsCfvK8KBM,2020-12-18 00:00:00
...,...,...,...,...
1316355491946213376,mailonline,2020-10-14 12:29:40,Your dog has a mid-life crisis too! Pet pooches 'get bored of life and less excited by new experiences when they hit middle age' https://t.co/0F7y1DI7ZH,2020-10-14 00:00:00
1316314715434430464,mailonline,2020-10-14 09:47:38,Thai pro-democracy protesters fight with royalists on streets of Bangkok amid calls to curb powers of billionaire king https://t.co/bQwqlTOq6w https://t.co/qrhX3DNhyz,2020-10-14 00:00:00
1316309906010583045,mailonline,2020-10-14 09:28:32,Fisherman is filmed holding dolphin calf underwater before dragging it under tarpaulin to be killed in Japan's Red Cove https://t.co/nFFrEpLNyP,2020-10-14 00:00:00
1316291670217052166,mailonline,2020-10-14 08:16:04,Thai pro-democracy protesters fight with royalists on streets of Bangkok amid calls to curb powers of billionaire king https://t.co/MxguCGd0xp,2020-10-14 00:00:00


# EDA - Exploratory Data Analysis 


In [7]:
# eda has also been performed in excel 
temp = df_import.sample(5)
[print(tweet,"\n") for tweet in temp.text.values]

The Electoral College meets Monday to cast votes for president and vice president, completing another key part of the election process that will eventually make Joe Biden the commander-in-chief next year. https://t.co/GISypasijF 

President Trump has four events planned in Pennsylvania today.
https://t.co/U7zhsltCWO 

“They’re not helping my family, they’re showing disrespect,” Wallace Sr. said. https://t.co/gdioYrwhRq 

Cheese, nuts and leafy vegetables could help ward off asthma, emphysema and even common colds https://t.co/NsKGpq7TZs 

The Supreme Court agreed to hear a case on whether the Trump administration can exclude undocumented immigrants from the calculations it will use in apportioning congressional seats.
https://t.co/9bVjgLl68R 



[None, None, None, None, None]

### Insights gained 


- there are many links in the tweets 
- unusual chars in written text
- trump, donald, donald trump are just examples of one rubric of synonyms that we want to consider as one for the topic features

# Preprocessing - 01 Cleaning of the text 

In [120]:
import emoji  # needs to be installed with pip
import regex # needs to be installed with pip
import re
import string
from collections import Counter

In [121]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    if not emoji_list==[] :
        print("",emoji_list,end="")
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):   
    '''
    Cleans text from urls
    '''
    text, _ = re.subn(r'http\S+', '', text)
    return text 

def url_free_text_s(text):  # as some https were missed out on by the previous function
    '''
    Cleans text from urls
    '''
    text, _ = re.subn(r'https\S+', '', text)
    return text  


# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

In [122]:
# Apply `call_emoji_free` which calls the function to remove all emoji's
# Apply 'url_free_text'which calls the function to remove all links

pd.options.mode.chained_assignment = None  # default='warn' turn off one the warning ... 

for orga in df_import.agency.unique():
    temp_df = df_import[df_import.agency ==orga]
    print(orga,"s tweets included the following emojis :")
    temp_df.text = temp_df.text.apply(call_emoji_free)
    print("\n")
    temp_df.text =temp_df.text.apply(url_free_text) 
    temp_df.text =temp_df.text.apply(url_free_text_s)
    df_import[df_import.agency ==orga] = temp_df
    
df_prep=df_import # df_prep = result of preprocessing steps

yahoonews s tweets included the following emojis :


huffpost s tweets included the following emojis :
 ['👇'] ['👀'] ['👌']

cnn s tweets included the following emojis :
 ['📩'] ['📩'] ['📩'] ['📩'] ['📩'] ['🎧'] ['🎧'] ['📩'] ['📩'] ['🎧'] ['📩'] ['📩'] ['📩'] ['▪', '▪', '▪', '▪'] ['📩'] ['📺', '📲', '🗨'] ['📩'] ['📩'] ['▪', '▪', '▪'] ['▪', '▪', '▪'] ['📩'] ['▪', '▪', '▪'] ['📩'] ['▪'] ['🎧'] ['🎧'] ['🎧'] ['📩'] ['📩'] ['🎧'] ['📩'] ['📩'] ['📩'] ['▪', '▪', '▪'] ['▪', '▪'] ['📩'] ['🎧'] ['📩'] ['⚡'] ['📩'] ['🎧'] ['📩'] ['🎧'] ['📩'] ['📩'] ['🎧'] ['▪', '▪', '▪', '▪', '▪', '▪'] ['▪', '▪', '▪', '▪', '▪', '▪'] ['🎃'] ['📩'] ['📩'] ['📩']

nytimes s tweets included the following emojis :
 ['🎄'] ['✨', '✨'] ['🎧'] ['👶'] ['🐀', '👨', '🍳', '🇫', '🇷'] ['🌕'] ['✨', '✨'] ['🦘', '💬', '🧍', '♂'] ['👇'] ['🎶', '🎶'] ['✅', '✅'] ['🐨', '🐨', '🐨', '🐨', '🐨', '🐨'] ['🕷'] ['☝'] ['☝'] ['☝'] ['☝'] ['☝'] ['⛰', '🐱', '⛰'] ['🐀', '🐀'] ['🗳'] ['👇'] ['🗳', '🗳', '🗳'] ['📌', '📌', '📌', '📌', '📌', '📌'] ['🐮'] ['🐶'] ['☕'] ['✅', '✅', '✅', '❓'] ['🗳', '🗳', '🗳'] ['🎶', '🎶'] ['💜'] ['

# Tokenization


#pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.0.0/en_core_web_md-2.0.0.tar.gz



In [123]:
# Load spacy
import spacy
import en_core_web_sm
#nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()

In [124]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', 
                    "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']


# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud 
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


In [125]:
n = 10
print("These are",n,"of our stopwords:\n")
for i in list(ALL_STOP_WORDS)[:n]:
    print(i)

These are 10 of our stopwords:

won't
am
seem
let's
myself
show
else
will
always
it's


In [126]:
tokens = []

for doc in tokenizer.pipe(df_prep.text,batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in ALL_STOP_WORDS:   #in the code that I was following, they only had stop_words .... 
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)
    
# Makes tokens column
df_prep['tokens'] = tokens

In [127]:
df_prep.sample(2)

,agency,created_at,text,date,tokens
1321057598104285185,huffpost,2020-10-27 11:54:10,The study showed just 4.4% of people in England had antibodies at the end of September.,2020-10-27 00:00:00,"[study, showed, 4.4%, people, england, antibodies, end, september.]"
1337091887728107528,nbcnews,2020-12-10 17:48:42,"Two members of a demolition crew remained missing Thursday following the collapse of a shuttered power plant in southern Ohio, authorities say.",2020-12-10 00:00:00,"[members, demolition, crew, remained, missing, thursday, following, collapse, shuttered, power, plant, southern, ohio,, authorities, say.]"


# Lemmatization and Synonyms

In [128]:
# more help: https://spacy.io/api/token
df_prep['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df_prep['tokens']]


# to remove weird characters and valueless words (in sense of our analysis),
# we want to remove words with len<=2
# to keep interesting words with this length, we define the following list

list_useful_2char_word = ["uk","us","ad"] 


def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False) and 
                (token.pos_!= 'PRON') and (token.is_digit== False) and 
                ((len(token)>2) or (token.text in list_useful_2char_word))): 
            lemmas.append(token.lemma_)
    
    return lemmas



In [129]:
# important this cell might take some minutes to run
# appllication of get_lemmas
df_prep['lemmas'] = df_prep['tokens_back_to_text'].apply(get_lemmas)


In [130]:
df_prep.sample(4) # visual inspection of lemmas

,agency,created_at,text,date,tokens,tokens_back_to_text,lemmas
1339542616090800129,cnn,2020-12-17 12:07:01,Here's a look at what we know about Moderna's coronavirus vaccine candidate and how it compares to the Pfizer-BioNTech vaccine that was authorized and shipped out to the first Americans earlier this week,2020-12-17 00:00:00,"[look, know, moderna's, coronavirus, vaccine, candidate, compares, pfizer-biontech, vaccine, authorized, shipped, americans, earlier, week]",look know moderna's coronavirus vaccine candidate compares pfizer-biontech vaccine authorized shipped americans earlier week,"[look, know, moderna, coronavirus, vaccine, candidate, compare, pfizer, biontech, vaccine, authorize, ship, americans, early, week]"
1334161913777893379,huffpost,2020-12-02 15:46:02,How to know when your cloth mask is no longer protecting you and others from the coronavirus.,2020-12-02 00:00:00,"[know, cloth, mask, longer, protecting, coronavirus.]",know cloth mask longer protecting coronavirus.,"[know, cloth, mask, long, protect, coronavirus]"
1339952328795938819,yahoonews,2020-12-18 15:15:04,BREAKING: Supreme Court rules challenge to Trump census plan is premature,2020-12-18 00:00:00,"[breaking:, supreme, court, rules, challenge, trump, census, plan, premature]",breaking: supreme court rules challenge trump census plan premature,"[breaking, supreme, court, rule, challenge, trump, census, plan, premature]"
1338596115692867586,foxnews,2020-12-14 21:25:58,"Outside groups converge on Georgia with bus tours, on-the-ground volunteers with Senate on the line",2020-12-14 00:00:00,"[outside, groups, converge, georgia, bus, tours,, on-the-ground, volunteers, senate, line]","outside groups converge georgia bus tours, on-the-ground volunteers senate line","[outside, group, converge, georgia, bus, tour, ground, volunteer, senate, line]"


In [131]:
## check if we still got tokens with length <= 2 that are not specified in out list
def too_short_lemmas(x):
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
                print(single_lemma,end=",")
                

def check_for_min_length_lemma(x):
    too_short=False
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
            too_short = True
    return too_short


print("There are still",
      len(df_prep[df_prep.lemmas.apply(lambda x:check_for_min_length_lemma(x))]),
         "tweets with lemmas of length <=2 that are not allowed.",
     "\nThese too short lemmas are:\n")

# check if these lemmas can be removed or if they include important information
print(df_prep.lemmas.apply(lambda x:too_short_lemmas(x)).values) 


# this happened because the token was for example going --> len(going)>2 but lemma = go

There are still 474 tweets with lemmas of length <=2 that are not allowed. 
These too short lemmas are:

go,go,go,go,go,go,go,go,go,go,go,go,go,90,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,gp,go,go,up,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,up,go,go,go,go,go,go,go,go,go,go,go,go,go,40,go,go,80,go,go,go,up,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,60,80,go,go,go,go,go,re,go,re,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,up,go,go,go,go,go,go,re,up,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,re,go,go,go,go,go,f,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,tv,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,up,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go

In [132]:
def remove_too_short(x):
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
                x.remove(single_lemma)
    return x

df_prep.lemmas = df_prep.lemmas.apply(lambda x:remove_too_short(x))

print("There are still",
      len(df_prep[df_prep.lemmas.apply(lambda x:check_for_min_length_lemma(x))]),
         "tweets with lemmas of length <=2 that are not allowed.")
# because of the small amount the effort to catch these slips was rated to high

There are still 7 tweets with lemmas of length <=2 that are not allowed.


In [133]:
# Make lemmas a string again
df_prep['lemmas_back_to_text'] = [' '.join(map(str,l)) for l in df_prep['lemmas']]


# handling of synonyms for topic based features
trump_synonyms = ["donald", "donald trump"] 
biden_synonyms = ["joe","joe biden"]
harris_synonyms = ["kamala","kamala harris"]
covid_synonyms = ["corona","covid-19","corona virus","virus","coronavirus"]

def replace_syn(repl_string,synonyms,col):
    for syn in synonyms:
        col_output = col.str.replace(syn,repl_string)
    return col_output
        
df_prep['lemmas_back_to_text'] = replace_syn("trump",trump_synonyms,df_prep['lemmas_back_to_text'])
df_prep['lemmas_back_to_text'] = replace_syn("biden",biden_synonyms,df_prep['lemmas_back_to_text'])
df_prep['lemmas_back_to_text'] = replace_syn("harris",harris_synonyms,df_prep['lemmas_back_to_text'])
df_prep['lemmas_back_to_text'] = replace_syn("covid",covid_synonyms,df_prep['lemmas_back_to_text'])


In [134]:
df_prep.sample(2)

,agency,created_at,text,date,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text
1316101437886279687,nytimes,2020-10-13 19:40:09,"He immigrated to the U.S. after his home country banned same-sex marriage. He later founded a homeless shelter for those seeking asylum. This new award is giving him $200,000 to help finance his endeavor.",2020-10-13 00:00:00,"[immigrated, u.s., home, country, banned, same-sex, marriage., later, founded, homeless, shelter, seeking, asylum., new, award, giving, $200,000, help, finance, endeavor.]","immigrated u.s. home country banned same-sex marriage. later founded homeless shelter seeking asylum. new award giving $200,000 help finance endeavor.","[immigrated, u.s, home, country, ban, sex, marriage, later, found, homeless, shelter, seek, asylum, new, award, give, 200,000, help, finance, endeavor]","immigrated u.s home country ban sex marriage later found homeless shelter seek asylum new award give 200,000 help finance endeavor"
1334903273162203137,foxnews,2020-12-04 16:51:56,Moderna reveals how long coronavirus immunity will last with their vaccine,2020-12-04 00:00:00,"[moderna, reveals, long, coronavirus, immunity, vaccine]",moderna reveals long coronavirus immunity vaccine,"[moderna, reveal, long, coronavirus, immunity, vaccine]",moderna reveal long covid immunity vaccine
1321897361556361216,cnn,2020-10-29 19:31:05,"After a string of dismal earnings and missed expectations last year, Tupperware is reporting soaring profits. The company announced it made $477.2 million in sales in the third quarter of 2020 — up 14% versus last year.",2020-10-29 00:00:00,"[string, dismal, earnings, missed, expectations, year,, tupperware, reporting, soaring, profits., company, announced, $477.2, million, sales, quarter, 2020, —, 14%, versus, year.]","string dismal earnings missed expectations year, tupperware reporting soaring profits. company announced $477.2 million sales quarter 2020 — 14% versus year.","[stre, dismal, earning, miss, expectation, year, tupperware, report, soar, profit, company, announce, 477.2, million, sale, quarter, versus, year]",stre dismal earning miss expectation year tupperware report soar profit company announce 477.2 million sale quarter versus year
1321267547958902784,nbcnews,2020-10-28 01:48:26,"The Texas Supreme Court has upheld Gov. Abbott's order limiting counties to 1 drop-off site for absentee ballots, dealing a blow to Democrats and voting rights groups that won a temporary injunction blocking the mandate.",2020-10-28 00:00:00,"[texas, supreme, court, upheld, gov., abbott's, order, limiting, counties, 1, drop-off, site, absentee, ballots,, dealing, blow, democrats, voting, rights, groups, won, temporary, injunction, blocking, mandate.]","texas supreme court upheld gov. abbott's order limiting counties 1 drop-off site absentee ballots, dealing blow democrats voting rights groups won temporary injunction blocking mandate.","[texas, supreme, court, uphold, gov, abbott, order, limit, county, drop, site, absentee, ballot, deal, blow, democrats, vote, right, group, win, temporary, injunction, block, mandate]",texas supreme court uphold gov abbott order limit county drop site absentee ballot deal blow democrats vote right group win temporary injunction block mandate
1318503496296038400,cnn,2020-10-20 10:45:04,QAnon may be a way for supporters to distract themselves from the failures of a President they see as the hero of a fight against villainy | Analysis by @donie,2020-10-20 00:00:00,"[qanon, way, supporters, distract, failures, president, hero, fight, villainy, |, analysis, @donie]",qanon way supporters distract failures president hero fight villainy | analysis @donie,"[qanon, way, supporter, distract, failure, president, hero, fight, villainy, analysis, @donie]",qanon way supporter distract failure president hero fight villainy analysis @donie


In [135]:
lemma_tokens_all = []
# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # first of the following does not seem to work - or is this bcs of string digits
    tokens = re.sub(r'[^a-zA-Z 0-9]', '', text) 
    tokens = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub(r'\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub(r'@*!*\$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') 
    tokens = tokens.strip('?') 
    tokens = tokens.strip('!') 
    tokens = tokens.strip("'") 
    tokens = tokens.strip(".") 


    tokens = tokens.lower().split() # Make text lowercase and split it
                
    lemma_tokens_all.extend(tokens)
    return tokens



In [136]:
# Apply tokenizer to df_all_orgas_cut_off and parallel create a list for every agency
# witl all lemma tokens

temp_series = pd.Series()
for i in df_prep.agency.unique():
    temp_df_part= df_prep[df_prep.agency==i]
    exec(f'list_{i}=[]') # create list to store all agency specific lemma tokens
    temp_df_part['lemma_tokens'] = temp_df_part['lemmas_back_to_text'].apply(tokenize)
    exec(f'list_{i}.extend(lemma_tokens_all)')
    lemma_tokens_all=[]
    temp_series= temp_series.append(temp_df_part['lemma_tokens'])

df_prep['lemma_tokens'] =temp_series

In [137]:
df_prep.head(2)

,agency,created_at,text,date,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
1340367556947943425,yahoonews,2020-12-19 18:45:03,"A $20,000 fee to change a wedding date? New bill aims to protect people from 'bad actor catering halls'",2020-12-19 00:00:00,"[$20,000, fee, change, wedding, date?, new, aims, protect, people, 'bad, actor, catering, halls']","$20,000 fee change wedding date? new aims protect people 'bad actor catering halls'","[20,000, fee, change, wedding, date, new, aim, protect, people, bad, actor, catering, hall]","20,000 fee change wedding date new aim protect people bad actor catering hall","[20,000, fee, change, wedding, date, new, aim, protect, people, bad, actor, catering, hall]"
1340329823282016256,yahoonews,2020-12-19 16:15:06,U.S. intelligence agencies are increasingly focused on domestic extremists. Their latest target: Satanists,2020-12-19 00:00:00,"[u.s., intelligence, agencies, increasingly, focused, domestic, extremists., latest, target:, satanists]",u.s. intelligence agencies increasingly focused domestic extremists. latest target: satanists,"[u.s, intelligence, agency, increasingly, focus, domestic, extremist, late, target, satanist]",u.s intelligence agency increasingly focus domestic extremist late target satanist,"[u.s, intelligence, agency, increasingly, focus, domestic, extremist, late, target, satanist]"


In [138]:
# inspect some lemma tokens for one of the agencies
n=40
print("first",n,"lemma tokens of yahoonews:")
print(*[i for i in list_yahoonews[:n]])

first 40 lemma tokens of yahoonews:
20,000 fee change wedding date new aim protect people bad actor catering hall u.s intelligence agency increasingly focus domestic extremist late target satanist rebuild trust justice department start end biden choice attorney general biden say lindsey graham personal disappointment recognize


In [144]:
# check how many tokens are left that do not comply to our length rule (if the number is small we keep them)
counter_01=0
for i in df_prep['lemma_tokens']:
    for k in i: 
        if (k.isnumeric()) or (len(k)<3) and (k  not in list_useful_2char_word):
            counter_01 +=1
            a=k
            print(k,"is of type",type(k),"and found in:",i,"\n")
print("There are only",counter_01,"lemma tokens left, that do not comply to our length rule")


ue is of type <class 'str'> and found in: ['story', 'futureproof', 'project', 'world', 'ready', 'crisis', 'series', 'collaboration', 'huffpost', 'ue', 'greenpeace', 'uk', 'journalism', 'project'] 

ue is of type <class 'str'> and found in: ['story', 'futureproof', 'project', 'world', 'ready', 'crisis', 'series', 'collaboration', 'huffpost', 'ue', 'greenpeace', 'uk', 'journalism', 'project'] 

1930 is of type <class 'str'> and found in: ['idea', 'gdp', 'value', 'good', 'service', 'exchange', 'country', 'come', '1930', 'period', 'characterize', 'soar', 'unemployment', 'deep', 'inequality', 'help', 'measure', 'country', 'recovery', 'great', 'depression'] 

ue is of type <class 'str'> and found in: ['story', 'futureproof', 'project', 'world', 'ready', 'crisis', 'series', 'collaboration', 'huffpost', 'ue', 'greenpeace', 'uk', 'journalism', 'project'] 

1960 is of type <class 'str'> and found in: ['fast', 'forward', 'century', 'u.s', '1960', 'basic', 'income', 'gathering', 'steam', 'martin',


#  create id2word and pre-process further 




In [194]:
# Create a gensim dictionary out of ALL lemma tokens
gensim_dict = Dictionary(df_prep['lemma_tokens'])


In [195]:
def some_stats_on_gen_dict(gd):
    n=60
    #This module implements the concept of a Dictionary 
    # – a mapping between words and their integer ids.
    print("The dict was created by lookint at",gd.num_docs,
          "tweets \nIt contains",len(gd),
          "different lemma tokens \n\nHere are the first",n,"tweets:")
    for i in range(n):
        print (gd.get(i),end=", ")

some_stats_on_gen_dict(gensim_dict)



The dict was created by lookint at 20796 tweets 
It contains 19025 different lemma tokens 

Here are the first 60 tweets:
20,000, actor, aim, bad, catering, change, date, fee, hall, new, people, protect, wedding, agency, domestic, extremist, focus, increasingly, intelligence, late, satanist, target, u.s, attorney, biden, choice, department, end, general, justice, rebuild, start, trust, disappointment, elect, graham, lindsey, personal, president, recognize, say, carolina, christmas, church, count, covid, musical, north, positive, test, 2-day, avoid, congress, covid-19, deal, elusive, remain, seek, shutdown, spending, 

In [196]:
# create a corpus with all final lemma tokens for each agency
lemma_tokens_all = []
for sublist in df_prep.agency.unique():
    exec(f'lemma_tokens_all.append(list_{sublist})')

print(len(lemma_tokens_all)) 

corpus_all = [gensim_dict.doc2bow(d) for d in lemma_tokens_all]

# create work frequency list for all agency 
word_frequencies = [[(gensim_dict[id], frequence) for id, 
                     frequence in couple] for couple in corpus_all]

7


# The following code creates the feature set for the bag of words approach

In [197]:
word_frequencies_sorted = []
for tuples in word_frequencies:
    word_frequencies_sorted.append(sorted(tuples, key=lambda tup: tup[1], reverse = True))


In [198]:
k = []
for word_frequencies_agency in word_frequencies_sorted:
    cumu_count = [0]
    counter = 1
    for i in word_frequencies_agency:
        cumu_count.append(cumu_count[counter-1]+i[1])
        counter+=1

    num_element = 0
    
    for i in cumu_count:
        if i > 0.2 * max(cumu_count):
            k.append(num_element)
            break
        num_element += 1

print(k)

[37, 63, 57, 63, 39, 51, 69]


In [199]:
agency = 0
feature_list = []
for j in k:
    for i in range(0,j):
        feature_list.append(word_frequencies_sorted[agency][i][0])
    agency +=1
    

unique_feature_list = list(set(feature_list))
print("Number of words across all agencies:",len(feature_list))
print("Number of unique words across all agencies:",len(unique_feature_list))

Number of words across all agencies: 379
Number of unique words across all agencies: 158


In [200]:
unique_feature_list

['positive',
 'show',
 'hunter',
 'trump',
 'russian',
 'black',
 'analysis',
 'accord',
 'biden',
 'senate',
 'change',
 'say',
 'democrats',
 'national',
 'woman',
 'sweden',
 'attack',
 'texas',
 'come',
 'covid-19',
 'right',
 'debate',
 'thousand',
 'gop',
 'covid',
 'york',
 'million',
 'case',
 'coney',
 'chinese',
 'warn',
 'body',
 'americans',
 'month',
 'democratic',
 'family',
 'mother',
 'late',
 'claim',
 'test',
 'supreme',
 'actor',
 'time',
 'expert',
 'france',
 'girl',
 'georgia',
 'story',
 'president',
 'long',
 'host',
 'take',
 'question',
 'early',
 'king',
 'court',
 'owner',
 'official',
 'death',
 'face',
 'campaign',
 'record',
 'man',
 'human',
 'life',
 'virus',
 'find',
 'people',
 'thursday',
 'times',
 'leave',
 'china',
 'live',
 'nov',
 'senator',
 'study',
 'look',
 'russia',
 'know',
 'terrorist',
 'macron',
 'new',
 'moment',
 'baby',
 'win',
 'election',
 'vaccine',
 'kill',
 'ballot',
 'hearing',
 'health',
 'breaking',
 'tuesday',
 'white',
 'he

In [201]:
dictOfWords = dict.fromkeys(unique_feature_list)
key_names = []
counter = 0
for key in dictOfWords:
    key_names.append(key)
    dictOfWords[key] = counter 
    counter+=1

lemma_list = list(df_prep.lemma_tokens)

len(lemma_list)
key_names.append('Not found')

In [202]:
feature_matrix = []
feature_size = len(unique_feature_list) + 1

lemma_list

for array in lemma_list:
    temp = [0]*feature_size
    for k in array:
        index = dictOfWords.get(k,feature_size-1)
        temp[index] = 1
    
    feature_matrix.append(temp)

print(feature_size)
print(len(feature_matrix))

159
20796


In [203]:
feature_df = pd.DataFrame(feature_matrix, index = df_prep.index, columns = key_names)
feature_df = feature_df.drop(['Not found'], axis=1)


In [204]:
feature_df.head(2)

,positive,show,hunter,trump,russian,black,analysis,accord,biden,senate,...,hit,elect,work,opinion,dec,big,nbcnewsthink,fox,europe,news
1340367556947943425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1340329823282016256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [205]:
combined_features = pd.concat([df_prep,feature_df],axis = 1)
combined_features.head(2)


,agency,created_at,text,date,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens,positive,...,hit,elect,work,opinion,dec,big,nbcnewsthink,fox,europe,news
1340367556947943425,yahoonews,2020-12-19 18:45:03,"A $20,000 fee to change a wedding date? New bill aims to protect people from 'bad actor catering halls'",2020-12-19 00:00:00,"[$20,000, fee, change, wedding, date?, new, aims, protect, people, 'bad, actor, catering, halls']","$20,000 fee change wedding date? new aims protect people 'bad actor catering halls'","[20,000, fee, change, wedding, date, new, aim, protect, people, bad, actor, catering, hall]","20,000 fee change wedding date new aim protect people bad actor catering hall","[20,000, fee, change, wedding, date, new, aim, protect, people, bad, actor, catering, hall]",0,...,0,0,0,0,0,0,0,0,0,0
1340329823282016256,yahoonews,2020-12-19 16:15:06,U.S. intelligence agencies are increasingly focused on domestic extremists. Their latest target: Satanists,2020-12-19 00:00:00,"[u.s., intelligence, agencies, increasingly, focused, domestic, extremists., latest, target:, satanists]",u.s. intelligence agencies increasingly focused domestic extremists. latest target: satanists,"[u.s, intelligence, agency, increasingly, focus, domestic, extremist, late, target, satanist]",u.s intelligence agency increasingly focus domestic extremist late target satanist,"[u.s, intelligence, agency, increasingly, focus, domestic, extremist, late, target, satanist]",0,...,0,0,0,0,0,0,0,0,0,0


In [206]:
# test whether the feature creation works

## entry number to test
test = 899
print(combined_features.iloc[test][6])
feature_df.iloc[test][feature_df.iloc[test]>0]


['wednesday', 'report', 'record', 'number', 'covid-19', 'case', 'hospitalization', 'death', 'ahead', 'key', 'meeting', 'country', 'second', 'covid-19', 'vaccine', 'green', 'light']


covid-19    1
case        1
death       1
record      1
vaccine     1
report      1
country     1
second      1
Name: 1339480974917300224, dtype: int64

In [207]:
assert len(combined_features) == len(df_import) 

In [208]:
## save file to drive
combined_features.to_csv("bag of words_02.csv")